# Method 1

## Load module

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import copy
import os

## Define parameter

In [ ]:
in_shape = (3, 224, 224)  # PyTorch expects (C, H, W)
batch_size = 32
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Prepare data

In [ ]:
# Paths

base_path = 'D:\Programming_File\Tensorflow_Files'
train_path = 'datasets/train_set'
val_path = 'datasets/val_set'
train_dir = os.path.join(base_path, train_path)
val_dir = os.path.join(base_path, val_path)
target_size = (64, 64)  # height, width in torchvision is (H, W)

print(f"Train directory exists: {train_dir}") if(os.path.exists(train_dir)) else print(f"Train directory does not exist: {train_dir}")
print(f"Validation directory exists: {val_dir}") if (os.path.exists(val_dir)) else print(f"Validation directory does not exist: {val_dir}")

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Load pre-trained ResNet50 without top layer

In [ ]:
base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
# Remove the final classification layer
num_ftrs = base_model.fc.in_features
base_model.fc = nn.Identity()  # no final fc, outputs feature vector

## Freeze base model layers

In [ ]:
for param in base_model.parameters():
    param.requires_grad = False

## Add custom classifier

In [ ]:
class TransferResNet50(nn.Module):
    def __init__(self, base_model, num_classes=1):
        super().__init__()
        self.base = base_model
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(num_ftrs, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
        self.sigmoid = nn.Sigmoid()  # for binary classification

    def forward(self, x):
        x = self.base(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
model = TransferResNet50(base_model).to(device)

## Loss & optimizer

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Training loop with validation

In [ ]:

""" 
If you simply did:
    best_model_wts = model.state_dict()
Then, best_model_wts would point to the same memory as the model’s parameters.
"""
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0
model_path = "transfer_resnet50_model.pth"

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Train
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.float().to(device)  # float for BCELoss
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        preds = (outputs > 0.5).long()
        running_corrects += torch.sum(preds == labels.long())

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)
    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.float().to(device)
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            preds = (outputs > 0.5).long()
            val_corrects += torch.sum(preds == labels.long())

    val_loss /= len(val_dataset)
    val_acc = val_corrects.double() / len(val_dataset)
    print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), model_path)
        print("✅ Best model saved!")

In [ ]:
print(model.state_dict().keys())
# dict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', ...])

## Load best weights after training

In [ ]:
model.load_state_dict(torch.load(model_path, map_location=device))

# Method 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader

# ===== Device =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===== Data augmentation =====
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('./datasets/train_set', transform=train_transforms)
val_dataset = datasets.ImageFolder('./datasets/val_set', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# ===== Base model (ResNet50 without top) =====
base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
for param in base_model.parameters():  # freeze
    param.requires_grad = False
num_ftrs = base_model.fc.in_features

# ===== Build new model like Sequential in Keras =====
model = nn.Sequential(
    base_model,
    nn.Flatten(),
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
).to(device)

# ===== Loss & optimizer =====
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ===== Training loop =====
for epoch in range(10):
    print(f"\nEpoch {epoch+1}/10")

    # --- Train ---
    model.train()
    train_loss, train_correct = 0.0, 0
    for x, y in train_loader:
        x, y = x.to(device), y.float().to(device)
        optimizer.zero_grad()
        preds = model(x).squeeze()
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
        train_correct += ((preds > 0.5).long() == y.long()).sum().item()

    train_acc = train_correct / len(train_dataset)
    print(f"Train Loss: {train_loss/len(train_dataset):.4f} Acc: {train_acc:.4f}")

    # --- Validation ---
    model.eval()
    val_loss, val_correct = 0.0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.float().to(device)
            preds = model(x).squeeze()
            loss = criterion(preds, y)

            val_loss += loss.item() * x.size(0)
            val_correct += ((preds > 0.5).long() == y.long()).sum().item()

    val_acc = val_correct / len(val_dataset)
    print(f"Val Loss: {val_loss/len(val_dataset):.4f} Acc: {val_acc:.4f}")

# ===== Save best model =====
torch.save(model.state_dict(), "transfer_resnet50_model.pth")
